In [35]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

True

In [36]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.2,
    max_output_tokens=1024,
    top_p=0.95,
    top_k=40,
    google_api_key=os.getenv("GEMINI_API_KEY")  # <-- Pass key here
)


In [37]:
#Create a state graph 
class LLMState(TypedDict):
    question: str
    answer: str

In [42]:
def llm_qa(state: LLMState) -> LLMState:
    #Extract the question from the state
    question = state['question']

    #Form the prompt
    prompt = f"Answer the following question: {question}"

    #Call the model
    answer = llm.invoke(prompt).content

    #Update the state with the answer
    state['answer'] = answer

    return state 

In [43]:
graph = StateGraph(LLMState)

graph.add_node('llm_qa',llm_qa)

graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

workflow = graph.compile()


In [ ]:
initial_state = LLMState(
    question="What is the capital of France?",
    answer=""   
)

final_state = workflow.invoke(initial_state)

print(final_state['answer']) 

Paris
